In [3]:
import urllib.request
import urllib
import requests
import lxml.html as lh
import pandas as pd
import csv
import numpy as np
import re

In [4]:
#getting the area codes from BLS reference csv
url = 'https://data.bls.gov/cew/doc/titles/area/area_titles.csv'
response = urllib.request.urlopen(url)
cr = csv.reader(response.read().decode('utf-8'))
area_code = list(cr)

while(['',''] in area_code) : 
    area_code.remove(['',''])
    
while([] in area_code) : 
    area_code.remove([])
    
area_code = np.array(area_code).reshape(-1, 2)
area_code = list(area_code.tolist())

In [5]:
len(area_code)

4724

In [6]:
#getting the statewide codes only

print(area_code[:10])

state_code = []
for s in area_code:
    if 'Statewide' in s[1] and 'Not Statewide' not in s[1]:
            state_code.append(s)

len(state_code)

[['area_fips', 'area_title'], ['US000', 'U.S. TOTAL'], ['USCMS', 'U.S. Combined Statistical Areas (combined)'], ['USMSA', 'U.S. Metropolitan Statistical Areas (combined)'], ['USNMS', 'U.S. Nonmetropolitan Area Counties (combined)'], ['01000', 'Alabama -- Statewide'], ['01001', 'Autauga County, Alabama'], ['01003', 'Baldwin County, Alabama'], ['01005', 'Barbour County, Alabama'], ['01007', 'Bibb County, Alabama']]


52

In [7]:
state_code_2 = []
for s in state_code:
    s[1] = re.sub(r" -- Statewide","", s[1])
    state_code_2.append(s)
    
len(state_code_2)
print(state_code_2)

[['01000', 'Alabama'], ['02000', 'Alaska'], ['04000', 'Arizona'], ['05000', 'Arkansas'], ['06000', 'California'], ['08000', 'Colorado'], ['09000', 'Connecticut'], ['10000', 'Delaware'], ['12000', 'Florida'], ['13000', 'Georgia'], ['15000', 'Hawaii'], ['16000', 'Idaho'], ['17000', 'Illinois'], ['18000', 'Indiana'], ['19000', 'Iowa'], ['20000', 'Kansas'], ['21000', 'Kentucky'], ['22000', 'Louisiana'], ['23000', 'Maine'], ['24000', 'Maryland'], ['25000', 'Massachusetts'], ['26000', 'Michigan'], ['27000', 'Minnesota'], ['28000', 'Mississippi'], ['29000', 'Missouri'], ['30000', 'Montana'], ['31000', 'Nebraska'], ['32000', 'Nevada'], ['33000', 'New Hampshire'], ['34000', 'New Jersey'], ['35000', 'New Mexico'], ['36000', 'New York'], ['37000', 'North Carolina'], ['38000', 'North Dakota'], ['39000', 'Ohio'], ['40000', 'Oklahoma'], ['41000', 'Oregon'], ['42000', 'Pennsylvania'], ['44000', 'Rhode Island'], ['45000', 'South Carolina'], ['46000', 'South Dakota'], ['47000', 'Tennessee'], ['48000', 

In [ ]:
#making dummy variable for ninth circuit and non-ninth-circuit states

state_merge = []
for s in state_code_2:
    s[0] = s[0]
    s[1] = 1 if s[1] in ['Alaska','Arizona','California','Hawaii','Idaho','Montana','Nevada','Oregon','Washington'] else 0
    state_merge.append(s)

In [ ]:
#getting years for API-available data

years = []
for x in range (2014,2019):
    years.append(str(x))

print(years)

['2014', '2015', '2016', '2017', '2018']


In [ ]:
#creating list of quarters
quarters = ["1","2","3","4"]

In [ ]:
#creating master list of quarters + state codes + years

import itertools
from itertools import product

arrays = [state_merge, years, quarters]
master_1 = list(product(*arrays))
print(master_1[:5])
master = [x for x in master_1 if not ('2018' in x[1] and '4' in x[2])]
len(master)
print(master[:5])


[(['01000', 0], '2014', '1'), (['01000', 0], '2014', '2'), (['01000', 0], '2014', '3'), (['01000', 0], '2014', '4'), (['01000', 0], '2015', '1')]
[(['01000', 0], '2014', '1'), (['01000', 0], '2014', '2'), (['01000', 0], '2014', '3'), (['01000', 0], '2014', '4'), (['01000', 0], '2015', '1')]


In [ ]:
# *******************************************************************************
# qcewCreateDataRows : This function takes a raw csv string and splits it into
# a two-dimensional array containing the data and the header row of the csv file
# a try/except block is used to handle for both binary and char encoding

def qcewCreateDataRows(csv):
    dataRows = []
    try: dataLines = csv.decode().split('\r\n')
    except er: dataLines = csv.split('\r\n');
    for row in dataLines:
        dataRows.append(row.split(','))
    return dataRows

In [ ]:
# *******************************************************************************
# qcewGetAreaData : This function takes a year, quarter, and area argument and
# returns an array containing the associated area data. Use 'a' for annual
# averages. 
# For all area codes and titles see:
# http://www.bls.gov/cew/doc/titles/area/area_titles.htm

def qcewGetAreaData(year,qtr,area):
    urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/[QTR]/area/[AREA].csv"
    urlPath = urlPath.replace("[YEAR]",year)
    urlPath = urlPath.replace("[QTR]",qtr.lower())
    urlPath = urlPath.replace("[AREA]",area.upper())
    httpStream = urllib.request.urlopen(urlPath)
    csv = httpStream.read()
    httpStream.close()
    return qcewCreateDataRows(csv)

In [ ]:
#example on how to pull data for Michgan '26000' for q1 of 2015
Michigan_Data = qcewGetAreaData('2015','1','26000')


In [ ]:
#loop to grab all the data
reg=[]
for item in master:
        year = item[1]
        qtr = item[2]
        area = item[0][0]
        data = qcewGetAreaData(year,qtr,area)
        reg.append(data)
        print(year)
        print(qtr)
        print(area)

2014
1
01000
2014
2
01000
2014
3
01000
2014
4
01000
2015
1
01000
2015
2
01000
2015
3
01000
2015
4
01000
2016
1
01000
2016
2
01000
2016
3
01000
2016
4
01000
2017
1
01000
2017
2
01000
2017
3
01000
2017
4
01000
2018
1
01000
2018
2
01000
2018
3
01000
2014
1
02000
2014
2
02000
2014
3
02000
2014
4
02000
2015
1
02000
2015
2
02000
2015
3
02000
2015
4
02000
2016
1
02000
2016
2
02000
2016
3
02000
2016
4
02000
2017
1
02000
2017
2
02000
2017
3
02000
2017
4
02000
2018
1
02000
2018
2
02000
2018
3
02000
2014
1
04000
2014
2
04000
2014
3
04000
2014
4
04000
2015
1
04000
2015
2
04000
2015
3
04000
2015
4
04000
2016
1
04000
2016
2
04000
2016
3
04000
2016
4
04000
2017
1
04000
2017
2
04000
2017
3
04000
2017
4
04000
2018
1
04000
2018
2
04000
2018
3
04000
2014
1
05000
2014
2
05000
2014
3
05000
2014
4
05000
2015
1
05000
2015
2
05000
2015
3
05000
2015
4
05000
2016
1
05000
2016
2
05000
2016
3
05000
2016
4
05000
2017
1
05000
2017
2
05000
2017
3
05000
2017
4
05000
2018
1
05000
2018
2
05000
2018
3
05000
2014
1
06000

2015
1
38000
2015
2
38000
2015
3
38000
2015
4
38000
2016
1
38000
2016
2
38000
2016
3
38000
2016
4
38000
2017
1
38000
2017
2
38000
2017
3
38000
2017
4
38000
2018
1
38000
2018
2
38000
2018
3
38000
2014
1
39000
2014
2
39000
2014
3
39000
2014
4
39000
2015
1
39000
2015
2
39000
2015
3
39000
2015
4
39000
2016
1
39000
2016
2
39000
2016
3
39000
2016
4
39000
2017
1
39000
2017
2
39000
2017
3
39000
2017
4
39000
2018
1
39000
2018
2
39000
2018
3
39000
2014
1
40000
2014
2
40000
2014
3
40000
2014
4
40000
2015
1
40000
2015
2
40000
2015
3
40000
2015
4
40000
2016
1
40000
2016
2
40000
2016
3
40000
2016
4
40000
2017
1
40000
2017
2
40000
2017
3
40000
2017
4
40000
2018
1
40000
2018
2
40000
2018
3
40000
2014
1
41000
2014
2
41000
2014
3
41000
2014
4
41000
2015
1
41000
2015
2
41000
2015
3
41000
2015
4
41000
2016
1
41000
2016
2
41000
2016
3
41000
2016
4
41000
2017
1
41000
2017
2
41000
2017
3
41000
2017
4
41000
2018
1
41000
2018
2
41000
2018
3
41000
2014
1
42000
2014
2
42000
2014
3
42000
2014
4
42000
2015
1
42000

In [ ]:
#checking how many records we got - should be 52 states times 19 = 988
len(reg)

988

In [ ]:
#the list above is too nested, now flattening the list
flat_list = []
for sublist in reg:
    for item in sublist:
        flat_list.append(item)
        
len(flat_list)
print(flat_list[:3])

[['"area_fips"', '"own_code"', '"industry_code"', '"agglvl_code"', '"size_code"', '"year"', '"qtr"', '"disclosure_code"', '"qtrly_estabs"', '"month1_emplvl"', '"month2_emplvl"', '"month3_emplvl"', '"total_qtrly_wages"', '"taxable_qtrly_wages"', '"qtrly_contributions"', '"avg_wkly_wage"', '"lq_disclosure_code"', '"lq_qtrly_estabs"', '"lq_month1_emplvl"', '"lq_month2_emplvl"', '"lq_month3_emplvl"', '"lq_total_qtrly_wages"', '"lq_taxable_qtrly_wages"', '"lq_qtrly_contributions"', '"lq_avg_wkly_wage"', '"oty_disclosure_code"', '"oty_qtrly_estabs_chg"', '"oty_qtrly_estabs_pct_chg"', '"oty_month1_emplvl_chg"', '"oty_month1_emplvl_pct_chg"', '"oty_month2_emplvl_chg"', '"oty_month2_emplvl_pct_chg"', '"oty_month3_emplvl_chg"', '"oty_month3_emplvl_pct_chg"', '"oty_total_qtrly_wages_chg"', '"oty_total_qtrly_wages_pct_chg"', '"oty_taxable_qtrly_wages_chg"', '"oty_taxable_qtrly_wages_pct_chg"', '"oty_qtrly_contributions_chg"', '"oty_qtrly_contributions_pct_chg"', '"oty_avg_wkly_wage_chg"', '"oty_av

In [ ]:
#sorting the flattened list and removing any duplicates
flat_list.sort()
new_list = list(flat_list for flat_list,_ in itertools.groupby(flat_list))
len(new_list)
array = np.asarray(new_list)
print(array[:2])

[list([''])
 list(['"01000"', '"0"', '"10"', '"50"', '"0"', '"2014"', '"1"', '""', '116272', '1827805', '1833319', '1849342', '19712038920', '8571763096', '191845414', '826', '""', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '""', '1521', '1.3', '17626', '1.0', '9259', '0.5', '10957', '0.6', '450001593', '2.3', '86303469', '1.0', '-50814078', '-20.9', '14', '1.7'])]


In [ ]:
#removing empty cells in the flattened list
array_2 = [x for x in array if not (x == [''])]

#indexing the last row of the flattened list, which is the column names that were copied in every iteration of the loop
index = array_2[-1]
array_3 = array_2[:-1]

#defining variables for the number of rows and number of columns in our dataset
l = len(array_3)
w = len(array_3[0])
print(l)
print(w)

2592683
42


In [22]:
#now using pandas to turn out dataset into a dataframe
#dataframe is more easily manipulated and merged with the ninth circuit data in 'state_merge'
matrix = np.reshape((array_3),(l,w))
df = pd.DataFrame(matrix, columns=index)
df.shape
df.head(3)

,"""area_fips""","""own_code""","""industry_code""","""agglvl_code""","""size_code""","""year""","""qtr""","""disclosure_code""","""qtrly_estabs""","""month1_emplvl""",...,"""oty_month3_emplvl_chg""","""oty_month3_emplvl_pct_chg""","""oty_total_qtrly_wages_chg""","""oty_total_qtrly_wages_pct_chg""","""oty_taxable_qtrly_wages_chg""","""oty_taxable_qtrly_wages_pct_chg""","""oty_qtrly_contributions_chg""","""oty_qtrly_contributions_pct_chg""","""oty_avg_wkly_wage_chg""","""oty_avg_wkly_wage_pct_chg"""
0,"""01000""","""0""","""10""","""50""","""0""","""2014""","""1""","""""",116272,1827805,...,10957,0.6,450001593,2.3,86303469,1.0,-50814078,-20.9,14,1.7
1,"""01000""","""0""","""10""","""50""","""0""","""2014""","""2""","""""",117167,1862944,...,14056,0.8,458068021,2.4,42738357,1.8,-11032523,-16.4,13,1.6
2,"""01000""","""0""","""10""","""50""","""0""","""2014""","""3""","""""",117243,1856219,...,24078,1.3,733360551,3.9,89509869,6.1,-4675632,-11.5,20,2.5


In [23]:
#now making another dataframe with the state codes and the dummy for whether it is Ninth Circuit or not
matrix_2 = np.reshape(state_merge,(52,2))
index_2 = ['state_code','ninth_circuit']
cir = pd.DataFrame(matrix_2, columns=index_2)
cir.head()

,state_code,ninth_circuit
0,01000,0
1,02000,1
2,04000,1
3,05000,0
4,06000,1


In [24]:
#using regex to get rid of the weird extra quotes around the area_fips values
df = df.replace({'"area_fips"': r'"'}, {'"area_fips"': ''}, regex=True)

#merging our dataframes so we gave a ninth-circuit value for each row
df_2 = pd.merge(df, cir, left_on='"area_fips"', right_on='state_code', how='outer')
df_2.head(3)

,"""area_fips""","""own_code""","""industry_code""","""agglvl_code""","""size_code""","""year""","""qtr""","""disclosure_code""","""qtrly_estabs""","""month1_emplvl""",...,"""oty_total_qtrly_wages_chg""","""oty_total_qtrly_wages_pct_chg""","""oty_taxable_qtrly_wages_chg""","""oty_taxable_qtrly_wages_pct_chg""","""oty_qtrly_contributions_chg""","""oty_qtrly_contributions_pct_chg""","""oty_avg_wkly_wage_chg""","""oty_avg_wkly_wage_pct_chg""",state_code,ninth_circuit
0,01000,"""0""","""10""","""50""","""0""","""2014""","""1""","""""",116272,1827805,...,450001593,2.3,86303469,1.0,-50814078,-20.9,14,1.7,01000,0
1,01000,"""0""","""10""","""50""","""0""","""2014""","""2""","""""",117167,1862944,...,458068021,2.4,42738357,1.8,-11032523,-16.4,13,1.6,01000,0
2,01000,"""0""","""10""","""50""","""0""","""2014""","""3""","""""",117243,1856219,...,733360551,3.9,89509869,6.1,-4675632,-11.5,20,2.5,01000,0


In [1]:
#the depressing part is we need data from 1990 to 2018, and the loop above only grabbed 2014 onward :(
#now writing code to bulk extract csvs from BLS zip files

import requests, zipfile, io

def qcewZip(year):
    urlPath = "https://data.bls.gov/cew/data/files/[YEAR]/csv/[YEAR]_qtrly_by_area.zip"
    urlPath = urlPath.replace("[YEAR]",year)
    r = requests.get(urlPath)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()

In [ ]:
years = []
for x in range (1990,1995):
    years.append(str(x))

for item in years:
    qcewZip(item)
        

In [ ]:
years_2 = []
for x in range (1995,2001):
    years_2.append(str(x))

for item in years_2:
    qcewZip(item)

In [ ]:
years_3 = []
for x in range (2001,2005):
    years_3.append(str(x))

for item in years_3:
    qcewZip(item)

In [ ]:
years_4 = []
for x in range (2005,2010):
    years_4.append(str(x))

for item in years_4:
    qcewZip(item)

In [2]:
years_5 = []
for x in range (2010,2014):
    years_5.append(str(x))

for item in years_5:
    qcewZip(item)

In [25]:
import os

In [26]:
os.getcwd()

'C:\\Users\\kaili\\OneDrive\\Desktop\\Research'

In [43]:
data2 = []

for root, dirs, files in os.walk(".", topdown=False):
   for name in files:
        cr = csv.reader(name.read().decode('utf-8'))
        data2.append(cr)
        print(os.path.join(root, name))

AttributeError: 'str' object has no attribute 'read'

In [41]:
len(data2)

0